In [1]:
import pandas as pd
import numpy as np
import re
from typing import List

In [2]:
df1 = pd.read_csv('/content/dataset/catalog_1.csv',sep=';')

In [3]:
df1.head(7)

,id,name
0,1,товар AAA модель 1 производитель DDD
1,2,товар AAA модель 2 производитель DDD
2,3,товар AAA модель 3 производитель DDD
3,4,товар AAA модель 4 производитель DDD
4,5,товар AAA модель 5 производитель DDD
5,6,товар AAA модель 6 производитель DDD
6,7,товар AAA модель 7 производитель DDD


In [4]:
df2 = pd.read_csv('/content/dataset/catalog_2.csv',sep=';')

In [5]:
df2.head(7)

,id,name,condition
0,1,товар AAb модель 5 производитель DD,некорректное наименование продукта и проводителя
1,2,AAA №6 DDD,смена шаблона описания товара
2,3,товар AAA модель 3 производитель DDD,добавлены лишние пробелы
3,4,производитель DDD товар AAA модель 4,перестановка слов в описании
4,5,товар ААА модель 7 производитель DDD,переход с латиницы на кириллицу
5,6,товар AAA модель 1 производитель DDD,полное соответствие значений
6,7,товар Aaa модель 2 производитель Ddd,изменен регистр букв


In [6]:
def str_lower(x:str)->str:
  '''Привести все символы к нижнему регистру'''
  return x.lower()

In [7]:
def del_space(x:str)->str:
  '''Удалить лишние пробелы и символы'''
  x_new=''
  x_new = re.sub(r'\s+', ' ', x)
  x_new = x_new.replace('№','')
  return x_new

In [8]:
def transform_russian_english(x:str)->str:
  '''Перевести названия с русского на английский'''
  x_new=''
  dict_russian_english = {'ааа':'aaa'}
  for current_key in dict_russian_english:
    if x.find(current_key)!=-1:
      x_new = x.replace(current_key,dict_russian_english.get(current_key))
    else:
      x_new = x
  return x_new

In [9]:
def transform_correct_name(x:str)->str:
  '''Привести наименования к одному типу'''
  x_new=''
  dict_correct_name = {r'\b[a]\w*\b':'aaa',r'\b[d]\w*\b':'ddd'}
  for current_key in dict_correct_name:
      x_new = re.sub(current_key, dict_correct_name.get(current_key), 
                     x if len(x_new)==0 else x_new)
  return x_new

In [10]:
df1['name'] = df1['name'].apply(lambda x: str_lower(x))
df2['name'] = df2['name'].apply(lambda x: str_lower(x))
df2['name'] = df2['name'].apply(lambda x: del_space(x))
df2['name'] = df2['name'].apply(lambda x: transform_russian_english(x))
df2['name'] = df2['name'].apply(lambda x: transform_correct_name(x))

In [11]:
df2.head(7)

,id,name,condition
0,1,товар aaa модель 5 производитель ddd,некорректное наименование продукта и проводителя
1,2,aaa 6 ddd,смена шаблона описания товара
2,3,товар aaa модель 3 производитель ddd,добавлены лишние пробелы
3,4,производитель ddd товар aaa модель 4,перестановка слов в описании
4,5,товар aaa модель 7 производитель ddd,переход с латиницы на кириллицу
5,6,товар aaa модель 1 производитель ddd,полное соответствие значений
6,7,товар aaa модель 2 производитель ddd,изменен регистр букв


In [12]:
def item_match(df1_col,df2_col)->List:
  '''Сопоставить строки согласно количеству найденных элементов'''
  list_number_match = []
  list_index_merge = []
  for current_string in df2_col:
    list_number_match.clear()
    for current_string_directory in df1_col:
      current_number = 0
      for current_element in current_string.split():
        if current_string_directory.find(current_element)!=-1:
          current_number = current_number + 1 
      list_number_match.append(current_number)
    list_index_merge.append(np.argmax(list_number_match))
  return list_index_merge

In [13]:
df2['id'] = list(map(lambda x: x+1,item_match(df1['name'],df2['name'])))

In [14]:
df2.head(7)

,id,name,condition
0,5,товар aaa модель 5 производитель ddd,некорректное наименование продукта и проводителя
1,6,aaa 6 ddd,смена шаблона описания товара
2,3,товар aaa модель 3 производитель ddd,добавлены лишние пробелы
3,4,производитель ddd товар aaa модель 4,перестановка слов в описании
4,7,товар aaa модель 7 производитель ddd,переход с латиницы на кириллицу
5,1,товар aaa модель 1 производитель ddd,полное соответствие значений
6,2,товар aaa модель 2 производитель ddd,изменен регистр букв


In [15]:
df_merge = df1.merge(df2, on=['id'], suffixes=['','_merge'])

In [16]:
df_merge.head(7)

,id,name,name_merge,condition
0,1,товар aaa модель 1 производитель ddd,товар aaa модель 1 производитель ddd,полное соответствие значений
1,2,товар aaa модель 2 производитель ddd,товар aaa модель 2 производитель ddd,изменен регистр букв
2,3,товар aaa модель 3 производитель ddd,товар aaa модель 3 производитель ddd,добавлены лишние пробелы
3,4,товар aaa модель 4 производитель ddd,производитель ddd товар aaa модель 4,перестановка слов в описании
4,5,товар aaa модель 5 производитель ddd,товар aaa модель 5 производитель ddd,некорректное наименование продукта и проводителя
5,6,товар aaa модель 6 производитель ddd,aaa 6 ddd,смена шаблона описания товара
6,7,товар aaa модель 7 производитель ddd,товар aaa модель 7 производитель ddd,переход с латиницы на кириллицу
